### Rastreo de noticias en Google News sobre temas de la Ciudad de México que competan o sean de interés del Instituto de Planeación de la Ciudad de México

El script extrae las notas de Google News de los temas de agua, movilidad, planeación, salud y seguridad de la Ciudad de México. El script extrae los títulos, url y fecha de las notas.

In [15]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
#Temas de interés
temas=["ipdp", "pgd", "pgot", "agua", "movilidad", "planeacion", "salud", "seguridad", "consulta", "ambiente", "cultura",  "asentamientos"]

In [32]:
#Función para extraer notas:
def get_notas(tema,periodo):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'
    }
    url = f'https://news.google.com/search?q={tema}%20ciudad%20de%20mexico%20when%3A{periodo}d&hl=es-419&gl=MX&ceid=MX%3Aes-419'
    response = requests.get(url, headers=headers)
    sopa=BeautifulSoup(response.text, 'html.parser')
    resultados=sopa.find_all('article')
    #Extraer títulos y url
    titulos=[]
    url=[]
    fechas=[]
    for i in range(len(resultados)):
        titulos.append(resultados[i].find('a', class_='JtKRv').text)
        url.append(resultados[i].find('a').get('href'))
        fecha = resultados[i].find('time')['datetime'] if resultados[i].find('time') else None
        fechas.append(fecha)
    # Eliminar el primer punto de la url
    url = [i[1:] for i in url]

    df=pd.DataFrame({'titulo':titulos,'url':url,'fecha_nota':fechas,'fecha_consulta':pd.to_datetime('today').strftime('%Y-%m-%d'),'tema':tema})
    df['url'] = 'https://news.google.com' + df['url'].astype(str)

    return df

In [35]:
#Tabla general de notas
consolidado_notas=pd.DataFrame()

#Ciclo para extraer las notas de ayer de los temas
for tema in temas:
    notas=get_notas(tema, 5)
    consolidado_notas=pd.concat([consolidado_notas,notas],axis=0)
print("Número total de notas encontradas", len(consolidado_notas))

Número total de notas encontradas 468


In [36]:
consolidado_notas["tema"].unique()

array(['ipdp', 'agua', 'movilidad', 'planeacion', 'salud', 'seguridad',
       'consulta', 'ambiente', 'cultura', 'asentamientos'], dtype=object)

In [43]:
consolidado_notas.to_excel("notas.xlsx", index=False)